In [33]:
import pandas as pd
import numpy as np
import seaborn as sea
import datetime as dt
from datetime import timedelta
url = "in.csv"

In [34]:
dataset = pd.read_csv(url, error_bad_lines=False)

In [35]:
selected = dataset[['cod_location','date_id','total_cases']]

In [36]:
selected = selected.sort_values('cod_location').groupby('cod_location').tail(5)

In [37]:
selected.head()

,cod_location,date_id,total_cases
86,0,20200326,75.0
85,0,20200325,42.0
84,0,20200324,40.0
83,0,20200323,34.0
87,0,20200327,75.0


In [38]:
#Input na rede neural
selected.to_csv('input.csv')

In [70]:
def aver(input):
    medias = input.groupby(['cod_location'])['total_cases'].mean()
    return medias

In [68]:
def diff(input):
    difference = input.groupby('cod_location')['total_cases'].apply(lambda x: x.tail(1).values[0] - x.head(1).values[0])
    return difference

In [73]:
diff(selected)

cod_location
0           0.0
1           8.0
2         452.0
3           2.0
4         -10.0
         ...   
207         0.0
208    867609.0
209       -37.0
210      5808.0
211       678.0
Name: total_cases, Length: 212, dtype: float64

In [ ]:
## Consertar ordenação dos dias no dataframe usado de parâmetro